## Required Modules

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.integrate import quad

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import random as rdInstituto 
import matplotlib.lines as mlines
from matplotlib.lines import Line2D
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm

import warnings

## Rescaled Hubble Parameter: $E(z)\equiv H(z)/H_0$

1. **$\Lambda$CDM** ($w_a=0$ and $w_0=-1$)
> $E^2(z) = \Omega_m (1+z)^3 + \Omega_{\Lambda}$
2. **$w_0$CDM** ($w_a=0$)
> $E^2(z) = \Omega_m (1+z)^3 + \Omega_{\Lambda} (1+z)^{3 (1+w_0)}$
3. **CPL**
> $E^2(z) = \Omega_m (1+z)^3 + \Omega_{\Lambda} (1+z)^{3 (1+w_0+w_a)} \exp \left[-\frac{3w_a z}{1+z}\right]$

For a flat universe we have: $\Omega_m + \Omega_{\Lambda} = 1 \Rightarrow \Omega_{\Lambda} = 1 - \Omega_m$

In [2]:
def function_E_z(zs, OmegaM, w0, wa):
    E2 = OmegaM * (1+zs)**3 + (1-OmegaM) * ((1+zs)**(3*(1+w0+wa))) * np.exp((-3*wa*zs)/(1+zs))
    return np.sqrt(E2)

## Fiducial Parameters

In [3]:
h = 0.7
OmegaM = 0.3
w0 = -1
wa = 0
c100t1 = 3.064

hh = 10**-6

Zl = 0
Zm = 0
zeta = 0

## Fisher Matrices

### - Redshift Drift

In [4]:
def FisherMatrix_z(zs, unc, h, OmegaM, w0, wa, NrYears, analysis):
    if analysis == 'redshift': 
        n = 4  
        F_z = np.zeros((n,n))
        for b in range(len(zs)):
            element = np.ones((n,n))
            for i in range(n):
                if i==0: 
                    deriv = dfdh_z(zs[b],h,OmegaM,w0,wa,NrYears)
                elif i==1:
                    deriv = dfdOmegaM_z(zs[b],h,OmegaM,w0,wa,NrYears)
                elif i==2:
                    deriv = dfdw0_z(zs[b],h,OmegaM,w0,wa,NrYears)
                else:
                    deriv = dfdwa_z(zs[b],h,OmegaM,w0,wa,NrYears)
                element[:,i]*=deriv
                element[i,:]*=deriv
            element*=unc[b]**-2
            F_z = F_z+element
        if wa == None:
            F_z = np.delete(F_z, 3, 1)
            F_z = np.delete(F_z, 3, 0)
        if w0 == None:
            F_z = np.delete(F_z, 2, 1)
            F_z = np.delete(F_z, 2, 0)

    return F_z

### - Alpha Variation: Bekenstein

In [5]:
def FisherMatrix_ab(zs, unc, OmegaM, w0, wa, Zl, Zm, OCP, analysis):
    #Assuming OmegaM as a constant
    if analysis == 'alpha' and OCP == 'constant':
        n = 2
        F_ab = np.zeros((n,n))
        for b in range(len(zs)):
            element = np.ones((n,n))
            for i in range(n):
                if i==0:
                    deriv = dfdZm(zs[b],OmegaM, w0, wa)
                elif i==1:
                    deriv = dfdZl(zs[b],OmegaM, w0, wa)
                element[:,i]*=deriv
                element[i,:]*=deriv
            element*=unc[b]**-2
            F_ab = F_ab+element
    
    #Assuming OmegaM as a parameter      
    if analysis == 'alpha' and OCP == 'parameter':
        n = 3
        F_ab = np.zeros((n,n))
        for b in range(len(zs)):
            element = np.ones((n,n))
            for i in range(n):
                if i==0:
                    deriv = dfdOmegaM_ab(zs[b], OmegaM, w0, wa, Zl, Zm)
                elif i==1:
                    deriv = dfdZm(zs[b],OmegaM, w0, wa, Zl, Zm)
                elif i==2:
                    deriv = dfdZl(zs[b],OmegaM, w0, wa, Zl, Zm)
                element[:,i]*=deriv
                element[i,:]*=deriv
            element*=unc[b]**-2
            F_ab = F_ab+element

    return F_ab

### - Alpha Variation: CPL

In [6]:
def FisherMatrix_aCPL(zs, unc, OmegaM, w0, wa, zeta, analysis):
    if analysis == 'alpha':
        n = 4  
        F_aCPL = np.zeros((n,n))
        for b in range(len(zs)):
            element = np.ones((n,n))
            for i in range(n):
                if i==0: 
                    deriv = dfdOmegaM_a(zs[b], OmegaM, w0, wa, zeta)
                elif i==1:
                    deriv = dfdw0_a(zs[b], OmegaM, w0, wa, zeta)
                elif i==2:
                    deriv = dfdwa_a(zs[b], OmegaM, w0, wa, zeta)
                else:
                    deriv = dfdzeta(zs[b], OmegaM, w0, wa, zeta)
                element[:,i]*=deriv
                element[i,:]*=deriv
            element*=unc[b]**-2
            F_aCPL = F_aCPL+element
  
    return F_aCPL

### - Add Separate Priors

In [7]:
def add1DPriors(F, priors): #to add priors locally in the intended parameters
    '''F-Fisher matrix, priors-[[sigma_prior1,local1],...]'''
    G = np.copy(F)
    for i in range(len(priors)): 
        G[priors[i][1],priors[i][1]]+=priors[i][0]**-2
    return G

## Ellipses Analysis

### - Marginalization

In [8]:
def marginalize(F,param,analysis):
    
    '''F-matrix, 'param'-parameter to be marginalized'''
    
    C=np.mat(F).I
    
    if analysis == 'redshift':
        if param=='h':
            a=0
        elif param=='OmegaM':
            a=1
        elif param=='w0':
            a=2
        elif param=='wa':
            a=3
        else:
            return 'invalid parameter'
    
    if analysis == 'alpha-BEK':
        if param=='OmegaM':
            a=0
        elif param=='Zm':
            a=1
        elif param=='Zl':
            a=2
        else:
            return 'invalid parameter'
        
    if analysis == 'alpha-CPL':
        if param=='OmegaM':
            a=0
        elif param=='w0':
            a=1
        elif param=='wa':
            a=2
        elif param=='zeta':
            a=3
        else:
            return 'invalid parameter'
        
    if analysis == 'red+alpha-BEK':
        if param=='h':
            a=0
        elif param=='OmegaM':
            a=1
        elif param=='Zm':
            a=2
        elif param=='Zl':
            a=3
        else:
            return 'invalid parameter'     
        
    if analysis == 'red+alpha-CPL':
        if param=='h':
            a=0
        elif param=='OmegaM':
            a=1
        elif param=='w0':
            a=2
        elif param=='wa':
            a=3
        elif param=='zeta':
            a=4
        else:
            return 'invalid parameter'       
        
 
    C=np.delete(C, a, 0)
    C=np.delete(C, a, 1)
    return np.asarray(np.mat(C).I)

### - Determination of the ellipses' parameters:

In [9]:
def preelipse(F):
    '''F-Fisher matrix 2*2
    Defining of the parameters of the ellipse:
    a2 - radius of the major axis
    b2 - radius of the minor axis
    theta - angle from positive x-axis to the ellipse’s major axis in the counterclockwise direction'''
    
    C=np.mat(F).I
    M2=(C[0,0]+C[1,1])/2.+np.sqrt((C[0,0]-C[1,1])**2/4.+C[0,1]**2)
    m2=(C[0,0]+C[1,1])/2.-np.sqrt((C[0,0]-C[1,1])**2/4.+C[0,1]**2)
    theta = 0.5*np.arctan((2*C[0,1])/(C[0,0]-C[1,1]))
    thetaDeg = theta*180/np.pi #conversion to degrees
    if C[0,0]>=C[1,1]:
        a2, b2=M2, m2
    else:
        a2, b2=m2, M2
    return np.sqrt(abs(a2)),np.sqrt(abs(b2)),thetaDeg

### - Drawing of the ellipses:

In [10]:
def ellipses(ells,axis):
    '''ells=[[x,y,da,db,angle(deg),L,color,style],...]; w0=0, E=1'''
    fig = plt.figure(figsize=[10, 8])
    ax = fig.add_subplot(111)
    Mx,My,XX,YY,XM,YM=0.,0.,[],[],ells[0][0],ells[0][1]
    n=len(ells)
    Is=[]
    
    #to draw the 3 confidence levels:
    #sigma=[1.52,2.48,3.44]
    
    Sigma=1.52  #1-sigma  
    
    for i in range(n):
        #for j in range(len(sigma)):
            #Sigma = sigma[j]
        
        d1=ells[i][2]*2*Sigma # major axis
        d2=ells[i][3]*2*Sigma # minor axis
        I=Ellipse((ells[i][0],ells[i][1]),d1,d2,ells[i][4])
        ax.add_artist(I)
        I.set_clip_box(ax.bbox)
        try:
            color=ells[i][6]
            style=ells[i][7]
            facecolor=ells[i][8]
            alpha=ells[i][9]
        except IndexError:
            color=(rd.random(),rd.random(),rd.random())
            style='-'
        I.set_facecolor(facecolor)
        I.set_edgecolor(color)
        I.set_linestyle(style)
        I.set_alpha(alpha)
        Mx, My= max(Mx,d1), max(My,d2)
        XX.append(ells[i][0]) 
        YY.append(ells[i][1])
        XM=max(XM,ells[i][0])
        YM=max(YM,ells[i][1])
        try:
            Is+=[mlines.Line2D([], [], color=color, linestyle=style, label=ells[i][5])]
        except IndexError:
            pass

    limX=(sum(XX)/n+(XM-sum(XX)/n)+.8*Mx,sum(XX)/n-(XM-sum(XX)/n)-.8*Mx)
    limY=(sum(YY)/n-(YM-sum(YY)/n)-.8*My,sum(YY)/n+(YM-sum(YY)/n)+.8*My)
    
    ax.set_xlim(min(limX),max(limX)) 
    ax.set_ylim(min(limY),max(limY)) 
    
    plt.xlabel(axis[0], fontsize=24)
    plt.ylabel(axis[1], fontsize=24)
    plt.tick_params(axis='both', which='major', labelsize=20)
 
    try:
        ells[i][5]
    except IndexError:
        pass

## Figures of Merit (FoM) and Correlation Coefficients $\rho$
### - Matching the parameters

In [11]:
def ParameterMatching(elem, analysis):
    
    '''Matching the parameters to their labels, numbers and values'''
    
    if analysis == 'redshift':
    
        if elem=='h' or elem==['h'] or elem==0:
            xx, num, short ='h', 0, 'h'   
        elif elem=='OmegaM' or elem==['Om'] or elem==['OmegaM'] or elem==1:
            xx, num, short = '$\Omega_m$', 1, 'OmegaM'
        elif  elem=='w0' or elem==['w0'] or elem==2:
            xx, num, short ='$w_{0}$', 2, 'w0'
        elif elem=='wa' or elem==['wa'] or elem==3:
            xx, num, short = '$w_{a}$' , 3, 'wa'
        else:
            raise NameError('This element cannot be marginalized.')  
            
    if analysis == 'alpha-CPL':
        
        if elem=='OmegaM' or elem==['Om'] or elem==['OmegaM'] or elem==0:
            xx, num, short = '$\Omega_m$', 0, 'OmegaM'
        elif  elem=='w0' or elem==['w0'] or elem==1:
            xx, num, short ='$w_{0}$', 1, 'w0'
        elif elem=='wa' or elem==['wa'] or elem==2:
            xx, num, short = '$w_{a}$' , 2, 'wa'
        elif elem=='zeta' or elem==['zeta'] or elem==3:
            xx, num, short = '$\zeta$' , 3, 'zeta'
        else:
            raise NameError('This element cannot be marginalized.')   
    
    if analysis == 'alpha-BEK':
        
        if elem=='OmegaM' or elem==['Om'] or elem==['OmegaM'] or elem==0:
            xx, num, short = '$\Omega_m$', 0, 'OmegaM'
        elif elem=='Zm' or elem==['Zm'] or elem==1:
            xx, num, short = '$\zeta_m$' , 1, 'Zm' 
        elif elem=='Zl' or elem==['Zl'] or elem==2:
            xx, num, short = '$\zeta_\Lambda$' , 2, 'Zl'
        else:
            raise NameError('This element cannot be marginalized.')
            
    if analysis == 'red+alpha-CPL':
        
        if elem=='h' or elem==['h'] or elem==0:
            xx, num, short ='h', 0, 'h'   
        elif elem=='OmegaM' or elem==['Om'] or elem==['OmegaM'] or elem==1:
            xx, num, short = '$\Omega_m$', 1, 'OmegaM'
        elif  elem=='w0' or elem==['w0'] or elem==2:
            xx, num, short ='$w_{0}$', 2, 'w0'
        elif elem=='wa' or elem==['wa'] or elem==3:
            xx, num, short = '$w_{a}$' , 3, 'wa'
        elif elem=='zeta' or elem==['zeta'] or elem==4:
            xx, num, short = '$\zeta$' , 4, 'zeta'
        else:
            raise NameError('This element cannot be marginalized.')
               
    if analysis == 'red+alpha-BEK':
        
        if elem=='h' or elem==['h'] or elem==0:
            xx, num, short ='h', 0, 'h'   
        elif elem=='OmegaM' or elem==['Om'] or elem==['OmegaM'] or elem==1:
            xx, num, short = '$\Omega_m$', 1, 'OmegaM'
        elif elem=='Zm' or elem==['Zm'] or elem==2:
            xx, num, short = '$\zeta_m$' , 2, 'Zm' 
        elif elem=='Zl' or elem==['Zl'] or elem==3:
            xx, num, short = '$\zeta_\Lambda$' , 3, 'Zl'
        else:
            raise NameError('This element cannot be marginalized.')
    
            
    return [xx,num,short]

### - Setting the order of parameters

In [12]:
def orderMargi(params, analysis):
    
    '''Setting the parameters to be marginalized in the specific order: h, OmegaM, w0, wa'''
    
    if analysis == 'redshift':
        AllParams=['h','OmegaM','w0','wa']
        
    if analysis == 'alpha-CPL':
        AllParams=['OmegaM','w0','wa','zeta']
        
    if analysis == 'alpha-BEK':
        AllParams=['OmegaM','Zm','Zl']
        
    if analysis == 'red+alpha-CPL':
        AllParams=['h','OmegaM','w0','wa','zeta']
        
    if analysis == 'red+alpha-BEK':
        AllParams=['h','OmegaM','Zm','Zl']
    
    
    NewParams=[]
    for i in AllParams:
        if i in params:
            NewParams.append(i)
    return NewParams[::-1]

### - FoM and $\rho$ determination

In [13]:
def FigMeriteRho(F,param,analysis): #param - param(s) which we are going to marginalize over
    
    try: #for a 5x5 matrix
        F[4][4]
        param = orderMargi(param,analysis) #Remove the parameters in the correct order
        param = [ParameterMatching(param[0],analysis)[2],ParameterMatching(param[1],analysis)[2],
                    ParameterMatching(param[2],analysis)[2]]
        F = marginalize(F,param[0],analysis)
        param = [param[1],param[2]]
            
    except IndexError:
        pass
        
    try: #for a 4x4 matrix
        F[3][3]
        param = [ParameterMatching(param[0],analysis)[2],ParameterMatching(param[1],analysis)[2]]
        F = marginalize(F,param[0],analysis)
        param = [param[1]]
            
    except IndexError:
        pass

    try: #for a 3x3 matrix
        F[2][2]
        param = ParameterMatching(param,analysis)[2]
        Fm = marginalize(F,param,analysis)
        
    except IndexError:
        Fm = np.copy(F)
            
    
    Pa = preelipse(Fm)
    C = np.mat(Fm).I
        
    FM = 1./(Pa[0]*Pa[1]*2.3) #1-sigma
    Rho = C[1,0]/np.sqrt(C[0,0]*C[1,1])
    
    
    warnings.warn('''Check the singularity of matrices! The precision of python may return values inconsistent with the numerical solution. To check if the matrices are singular use the function SingularMatrix(Fm).''')
        
    return (FM,Rho) 

In [14]:
def SingularMatrix(Fm):
    
    Matrix1 = np.mat(Fm).I
    Matrix2 = Matrix1*Fm
    
    T = "If the matrix above is not the identity, the matrix analysed is singular."
    
    
    return(Matrix2, T)

## 1$\sigma$ Uncertainty

In [15]:
def Sigmas(F):
    
    sigmas = np.zeros(len(F))
    Fm = np.copy(F)
    C = np.mat(Fm).I
    for i in range(len(F)):
        sigmas[i] = np.sqrt(C[i,i])
        
    return sigmas